In [1]:
import torch
from transformers import AutoTokenizer, BlenderbotForConditionalGeneration


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
mname = "facebook/blenderbot-400M-distill"
weights_path = "pruned_400m_blender_bot2.pt"

model = BlenderbotForConditionalGeneration.from_pretrained(mname)
print(model.state_dict)
model.load_state_dict(torch.load(weights_path))
print(model.state_dict)
tokenizer = AutoTokenizer.from_pretrained(mname)
UTTERANCE = "I'm looking for someone to talk to. Life can be so lonely sometimes and it helps to have someone to vent to."
print("Human:", UTTERANCE)

<bound method Module.state_dict of BlenderbotForConditionalGeneration(
  (model): BlenderbotModel(
    (shared): Embedding(8008, 1280, padding_idx=0)
    (encoder): BlenderbotEncoder(
      (embed_tokens): Embedding(8008, 1280, padding_idx=0)
      (embed_positions): BlenderbotLearnedPositionalEmbedding(128, 1280)
      (layers): ModuleList(
        (0): BlenderbotEncoderLayer(
          (self_attn): BlenderbotAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_fe

In [4]:
inputs = tokenizer([UTTERANCE], return_tensors="pt")
reply_ids = model.generate(**inputs)
print("Bot:", tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

/opt/homebrew/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 60 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Bot:  I'm sorry to hear that. I'm here for you if you need someone to chat with.


In [5]:
model.config

BlenderbotConfig {
  "_name_or_path": "facebook/blenderbot-400M-distill",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "BlenderbotForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1280,
  "decoder_attention_heads": 32,
  "decoder_ffn_dim": 5120,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 32,
  "encoder_ffn_dim": 5120,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_no_repeat_ngram_size": 3,
  "eos_token_id": 2,
  "extra_layer_norm": false,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
 

In [10]:
from transformers.models.blenderbot.configuration_blenderbot import BlenderbotConfig
from pruning.pruned_blender_bot import PrunedBlenderbotForConditionalGeneration

weights_path = "pruned_400m_blender_bot2.pt"
state_dict_shapes_path = "pruned_400m_blender_bot2_state_dict_shapes.pkl"

blenderbot_config = BlenderbotConfig(
    vocab_size=8008,
    max_position_embeddings=128,
    encoder_layers=2,
    encoder_ffn_dim=5120,
    encoder_attention_heads=32,
    decoder_layers=12,
    decoder_ffn_dim=5120,
    decoder_attention_heads=32,
    encoder_layerdrop=0.0,
    decoder_layerdrop=0.0,
    layernorm_variant="prelayernorm",
    use_cache=True,
    is_encoder_decoder=True,
    activation_function="gelu",
    d_model=1280,
    dropout=0.1,
    attention_dropout=0.0,
    activation_dropout=0.0,
    init_std=0.02,
    decoder_start_token_id=1,
    scale_embedding=False,
    pad_token_id=0,
    bos_token_id=1,
    eos_token_id=2,
    encoder_no_repeat_ngram_size=3,
    forced_eos_token_id=2,
    num_beams=10,
    length_penalty=65,
    min_length=20,
    static_position_embeddings=False,
)

In [14]:
#pruned_model = PrunedBlenderbotForConditionalGeneration(blenderbot_config, state_dict_shapes_path)
pruned_model = BlenderbotForConditionalGeneration(blenderbot_config)
pruned_model.load_state_dict(torch.load(weights_path))

<All keys matched successfully>

In [15]:
reply_ids = pruned_model.generate(**inputs)
print("Bot:", tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

Bot:   a to. to to to. to to to a to to to to to to


In [17]:
print(pruned_model.state_dict())

OrderedDict([('final_logits_bias', tensor([[0., 0., 0.,  ..., 0., 0., 0.]])), ('model.shared.weight', tensor([[ 0.0117, -0.0016, -0.0037,  ...,  0.0047, -0.0064, -0.0061],
        [-0.0008, -0.0297,  0.0005,  ..., -0.0311,  0.0016, -0.0305],
        [-0.0167,  0.0146, -0.0097,  ..., -0.0175,  0.0177, -0.0497],
        ...,
        [ 0.0285, -0.0268,  0.0322,  ...,  0.0152,  0.0374, -0.0690],
        [-0.0064, -0.0241,  0.0508,  ..., -0.0005,  0.0751, -0.0540],
        [ 0.0312,  0.0444,  0.0149,  ...,  0.0448,  0.0242, -0.0059]])), ('model.encoder.embed_tokens.weight', tensor([[ 0.0117, -0.0016, -0.0037,  ...,  0.0047, -0.0064, -0.0061],
        [-0.0008, -0.0297,  0.0005,  ..., -0.0311,  0.0016, -0.0305],
        [-0.0167,  0.0146, -0.0097,  ..., -0.0175,  0.0177, -0.0497],
        ...,
        [ 0.0285, -0.0268,  0.0322,  ...,  0.0152,  0.0374, -0.0690],
        [-0.0064, -0.0241,  0.0508,  ..., -0.0005,  0.0751, -0.0540],
        [ 0.0312,  0.0444,  0.0149,  ...,  0.0448,  0.0242, -